# Run stock-price prediction model

## Notebook ([`papermill`](https://papermill.readthedocs.io/en/latest/index.html)) parameters

In [1]:
run_time = '2019-08-01T10:00'
stop_minutes = 10
dir = None
earliest_data_cutoff = '2019-04-01'
ticker = 'AAPL'
batch_size = 100000
max_epochs = 10000

## Date/Time helpers

In [2]:
from sys import executable as python
from datetime import datetime as dt, date, time, timedelta as Δ
from dateutil.parser import parse

strptime = dt.strptime
now = dt.now()
today = now.date()

if run_time:
    run_time = parse(run_time)
else:
    run_time = dt.now()

run_date = run_time.date()

if stop_minutes:
    stop_at = run_time + Δ(minutes=stop_minutes)
else:
    stop_at = None

earliest_data_cutoff = parse(earliest_data_cutoff)
earliest_data_cutoff_date = earliest_data_cutoff.date()
    
print('Running at %s (stopping within %dmins)' % (run_time.strftime('%Y-%m-%d %H:%M:%S'), stop_minutes))

Running at 2019-08-01 10:00:00 (stopping within 10mins)


## Initialize `data`, `models` directories

In [3]:
from pathlib import Path
if not dir:
    dir = Path.cwd()

data_dir = dir / 'data'
data_dir.mkdir(parents=True, exist_ok=True)
models_dir = dir / 'models'

## Read IEX API credentials
Search for a secret key in:
- `$IEX_SECRET_KEY` env var
- `iex.secret_key` in a config file located at:
  - `$CONFIG_PATH` if set
  - `~/.config/iex.ini` otherwise

In [4]:
from os import environ as env

api = 'https://cloud.iexapis.com'

SECRET_KEY_VAR = 'IEX_SECRET_KEY'
CONFIG_PATH_VAR = 'CONFIG_PATH'

if SECRET_KEY_VAR in env:
    secret_key = env[SECRET_KEY_VAR]
else:
    if CONFIG_PATH_VAR in env:
        config_path = Path(env[CONFIG_PATH_VAR])
    else:
        config_path = Path.home() / '.config' / 'iex.ini'

    if not config_path.exists():
        raise Exception("No %s env var, and config path %s doesn't exist" % (SECRET_KEY_VAR, config_path))

    from configparser import ConfigParser
    config = ConfigParser()
    config.read(str(config_path))
    iex_config = config['iex']
    secret_key = iex_config['secret_key']

## Initialize `dates` and `minutes` that trading occurs during

In [5]:
start_minute = time(9, 30)
end_minute = time(16, 0)
def get_minutes():
    minute = start_minute
    while minute < end_minute:
        yield minute
        hr = minute.hour
        min = minute.minute
        min += 1
        if min == 60:
            min = 0
            hr += 1
        minute = time(hr, min)

minutes = list(get_minutes())
num_minutes = len(minutes); num_minutes

def get_dates(start_date, end_date, step=1):
    date = start_date
    while date != end_date:
        # only emit weekdays
        if date.weekday() <= 4:
            yield date
        date += Δ(days=step)

start_date = earliest_data_cutoff_date
end_date = run_date + Δ(days=1)
dates = list(get_dates(start_date, end_date))
num_dates = len(dates)

print(
    'Fetching/Processing data from %d days ([%s,%s)), %d minutes per day' % (
        num_dates, 
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d'),
        num_minutes,
    )
)

Fetching/Processing data from 89 days ([2019-04-01,2019-08-02)), 390 minutes per day


In [6]:
def get_ticker_dir(ticker):
    return data_dir / ticker

def get_ticker_date_path(ticker, date):
    return get_ticker_dir(ticker) / date.strftime('%Y%m%d')

In [7]:
!{python} -m pip install -Uq requests
from requests import get as GET

In [8]:
import json

def fetch(date, ticker, refetch_partial=False):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    refetch = False
    prev_data = None
    if out_path.exists():
        if refetch_partial:
            with out_path.open('r') as f:
                prev_data = json.load(f)
                if len(prev_data) < num_minutes:
                    refetch = True
                    print(
                        'Re-fetching data for %s from %s (found %d per-minute quotes instead of %d)' % (
                            ticker, 
                            date_str, 
                            len(prev_data), 
                            num_minutes
                        )
                    )
                else:
                    return True
        else:
            return True
    else:
        print('Fetching data for %s from %s' % (ticker, date_str))

    url = f'https://cloud.iexapis.com/stable/stock/{ticker}/chart/date/{date_str}?token={secret_key}'
    resp = GET(url)
    resp.raise_for_status()
    data = json.loads(resp.content)
    if prev_data is None or len(data) > len(prev_data):
        with out_path.open('wb') as f:
            f.write(resp.content)

        if refetch:
            print('Re-fetch found data for %s %s' % (date_str, ticker))
        return True
    elif len(data) < len(prev_data):
        raise Exception('Found %d data, less than previous amount %d' % (len(data), len(prev_data)))
    else:
        print('Re-fetched %s, found same %d data' % (out_path, len(data)))

    return False

In [9]:
!{python} -m pip install -Uq joblib
from joblib import Parallel, delayed

Fetch quotes between a start point in the past (IEX seems to serve 6-7mos of historic data) and today

In [10]:
%%time

N = 32  # fetch parallelism
refetch_empty = False

Parallel(n_jobs=N)(
    delayed(fetch)(date, ticker, refetch_partial=refetch_empty or date == today)
    for date in dates 
); None

CPU times: user 453 ms, sys: 197 ms, total: 650 ms
Wall time: 8.05 s


In [11]:
!{python} -m pip install -Uq pandas matplotlib numpy scipy
import numpy as np
from numpy import \
    array, \
    nan, isnan as na, \
    zeros, \
    count_nonzero as cnz, \
    mean, std, \
    unique, \
    logical_and as l_and, \
    logical_or as l_or, \
    exp, log, sqrt
from numpy.random import shuffle, permutation
from pandas import concat, DataFrame as DF, read_csv, read_json
import pandas as pd
from scipy.stats import describe

These are the columns that we receive from IEX, describing trades that took place there:

In [12]:
features = [ 'open', 'close', 'high', 'low', 'average', 'volume', 'notional', 'numberOfTrades' ]

In [13]:
def load_ticker_date_df(date, ticker, latest_data_datetime=None):
    date_str = date.strftime('%Y%m%d')
    out_path = get_ticker_date_path(ticker, date)
    if not out_path.exists():
        return None
    df = read_json(out_path)
    if df.empty:
        return None
    # Convert "date", "minute" columns into a single "datetime" column
    df['datetime'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d')) + ' ' + df['minute']
    df['datetime'] = df['datetime'].apply(lambda s: strptime(s, '%Y-%m-%d %H:%M'))
    df.drop(columns=['date', 'minute'])
    if latest_data_datetime:
        df = df.loc[df.datetime <= latest_data_datetime]
    
    # Drop other columns ("market"-data columns, which are updated on a 15-min delay)
    df = df[['datetime'] + features]

    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)

    return df

In [14]:
def load_ticker_df(ticker, N=None, limit=None):
    if limit is None:
        ds = dates
    elif type(limit) == int:
        ds = dates[:limit]
    elif type(limit) == date:
        ds = [ date for date in dates if date < limit ]        
    elif type(limit) == dt:
        ds = [ date for date in dates if date < limit.date() ]
    else:
        raise Exception('Unrecognized limit: %s' % limit)

    if N is None:
        df = concat([ load_ticker_date_df(date, ticker, latest_data_datetime=run_time) for date in ds ])
    else:
        df = concat(Parallel(n_jobs=N)( delayed(load_ticker_date_df)(date, ticker) for date in ds ))
    
    for col in features:
        # "quantity" features (volume, numberOfTrades, notional) are sometimes -1 instead of NaN
        df[col] = df[col].apply(lambda n: nan if n < 0 else n)

    return df

In [15]:
%%time
aapl = load_ticker_df('AAPL'); aapl

CPU times: user 2.6 s, sys: 47.7 ms, total: 2.65 s
Wall time: 2.71 s


,open,close,high,low,average,volume,notional,numberOfTrades
datetime,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.600,191.189,4320,825935.940,44
2019-04-01 09:31:00,190.700,190.98,190.980,190.640,190.761,3246,619210.510,32
2019-04-01 09:32:00,191.060,190.93,191.090,190.780,190.951,2253,430211.740,30
2019-04-01 09:33:00,190.980,190.83,191.010,190.760,190.946,2241,427911.290,27
2019-04-01 09:34:00,190.760,190.70,190.760,190.600,190.666,1069,203822.465,12
...,...,...,...,...,...,...,...,...
2019-08-01 09:56:00,214.685,214.79,214.860,214.685,214.751,3966,851703.860,41
2019-08-01 09:57:00,214.800,214.96,214.980,214.770,214.900,3028,650717.425,38
2019-08-01 09:58:00,214.980,214.91,215.040,214.910,215.009,5451,1172014.895,47


## Fill missing data
Minutes where no trades were recorded can be assigned prices based on previous minutes' closing prices

We don't attempt to cross day boundaries, as the moves overnight are often much larger than minute-to-minute moves during the day, and skew the data.

Make a series representing minutes that are the first of their day (i.e. 9:30am)

In [16]:
prev_datetime = aapl.index.to_series().shift(1)
datetime = aapl.index.to_series()

def to_date(dt):
    return dt.date()

day_start = datetime.apply(to_date) != prev_datetime.apply(to_date)

Populate each minute with the previous minute's closing price

Assume each day's initial opening is the same as an imagined closing price from a minute prior (when trading wasn't open).

In [17]:
prev_close = aapl.close.shift(1)
prev_close[day_start] = aapl.open
aapl['prev_close'] = prev_close

Track minutes where no trades occurred:

In [18]:
nan_idxs = aapl[features].isna().any(axis=1); nan_idxs
nans = aapl[nan_idxs]
nans

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,193.000
2019-04-05 13:36:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.540
2019-04-05 14:29:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,196.795
2019-04-15 13:33:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,198.910
2019-04-16 13:39:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,199.725
...,...,...,...,...,...,...,...,...,...
2019-07-29 13:11:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,210.070
2019-07-29 13:43:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,210.135
2019-07-29 14:15:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,210.230


Spot-check: consecutive pairs of minutes where no trades occurred

In [19]:
nan_pairs = l_and(nan_idxs, nan_idxs.shift(1).fillna(False))
concat([ aapl[nan_pairs], aapl[nan_pairs.shift(-1).fillna(False)] ]).sort_index()

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-07-24 13:00:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,208.12
2019-07-24 13:01:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN
2019-07-24 13:24:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,207.90
2019-07-24 13:25:00,NaN,NaN,NaN,NaN,NaN,0,0.0,0,NaN


Group by date, and forward-fill "previous closing" price.

For minutes when no trades occurred, we will treat the last valid closing price as both the opening and closing price. However, we don't persist these across day boundaries (this results in apparent large jumps at opening that skew training).

In [20]:
grouped = aapl.groupby(to_date)
aapl.prev_close = grouped.prev_close.apply(lambda s: s.fillna(method='ffill'))
aapl.count()

open              30584
close             30584
high              30584
low               30584
average           30584
volume            30658
notional          30658
numberOfTrades    30658
prev_close        30658
dtype: int64

Fill `NaN` openings with the previous valid closing price, which is interpreted as the start and end price for a minute where no activity occurred:

In [21]:
fill_cols = [ 'open', 'close', 'high', 'low', 'average', ]
for col in fill_cols:
    aapl[col].fillna(aapl.prev_close, inplace=True)
aapl.count()

open              30658
close             30658
high              30658
low               30658
average           30658
volume            30658
notional          30658
numberOfTrades    30658
prev_close        30658
dtype: int64

We expect that there are no remaining `NaN`s in the dataset (unless they appear during the first minute of the day; at the time of writing that has not been observed)

Spot check that `NaN` entries have been filled in based on the previous minute's closing price:

In [22]:
aapl[l_or(nan_idxs, nan_idxs.shift(-1).fillna(False))]

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close
datetime,,,,,,,,,
2019-04-02 13:28:00,192.990,193.000,193.05,192.990,193.014,1100,212316.000,11,192.990
2019-04-02 13:29:00,193.000,193.000,193.00,193.000,193.000,0,0.000,0,193.000
2019-04-05 13:35:00,196.535,196.540,196.54,196.535,196.537,205,40290.200,3,196.495
2019-04-05 13:36:00,196.540,196.540,196.54,196.540,196.540,0,0.000,0,196.540
2019-04-05 14:28:00,196.820,196.795,196.82,196.795,196.811,731,143868.985,13,196.820
...,...,...,...,...,...,...,...,...,...
2019-07-29 14:15:00,210.230,210.230,210.23,210.230,210.230,0,0.000,0,210.230
2019-07-29 14:18:00,210.110,210.090,210.11,210.090,210.100,400,84040.000,4,210.150
2019-07-29 14:19:00,210.090,210.090,210.09,210.090,210.090,0,0.000,0,210.090


## Log-fold transforms

Market-data time-series are frequently approximated as being log-normally distributed; the fold-change from one interval to the next is considered to be normally distributed.

Here we copy the price data above, make `open` a log-fold change over the previous `close`, and express other intra-minute features as a log-fold change over their corresponding `open`ing price:

In [23]:
lg = aapl.copy()
for col in fill_cols[1:]:
    lg[col] = log(lg[col] / lg.open)
lg.open = log(lg.open / lg.prev_close)

lg.drop(columns='prev_close', inplace=True)
lg[fill_cols][l_or(nan_idxs, nan_idxs.shift(1).fillna(False))]

,open,close,high,low,average
datetime,,,,,
2019-04-02 13:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-02 13:30:00,0.000414,0.000000,0.000362,-0.000052,0.000104
2019-04-05 13:36:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-04-05 13:37:00,-0.000102,-0.000051,0.000000,-0.000051,-0.000010
2019-04-05 14:29:00,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...
2019-07-29 14:16:00,-0.000523,0.000190,0.000190,0.000000,0.000081
2019-07-29 14:19:00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-07-29 14:20:00,0.000167,-0.000071,0.000000,-0.000119,-0.000095


In [24]:
lg.count()

open              30658
close             30658
high              30658
low               30658
average           30658
volume            30658
notional          30658
numberOfTrades    30658
dtype: int64

Normalize and rename some features:
- `trades` ⟶ `sqrt(numberOfTrades)` (variance-stabilizing transform for a Poisson distribution)
- `volume` ⟶ `avgVol` (divide out then number of trades, and `log`-transform to attempt to normalize outliers

In [25]:
lg.drop(columns=[ 'volume', 'notional', 'numberOfTrades' ], inplace=True)
lg['trades'] = sqrt(aapl.numberOfTrades)  # variance-stabilizing transform for a Poisson distribution
lg['avgVol'] = log(1 + (aapl.volume / aapl.numberOfTrades).fillna(0))  # volume-per-trade  seems extremely high-tailed, so apply log1p
lg.rename(columns={'average': 'avg', 'high': 'hi', 'low': 'lo'}, inplace=True)
lg

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735
...,...,...,...,...,...,...,...
2019-08-01 09:56:00,0.000023,0.000489,0.000815,0.000000,0.000307,6.403124,4.582226
2019-08-01 09:57:00,0.000047,0.000745,0.000838,-0.000140,0.000465,6.164414,4.390543
2019-08-01 09:58:00,0.000093,-0.000326,0.000279,-0.000326,0.000135,6.855655,4.761992


Add `next_close` to each minute, which is the log-fold change of the next-minute's closing price over the current minute's.

This will be treated as the "label" for a training sample anchored at each minute: given the current minute's activity (and that of several minutes prior), can we predict the next minute's closing price?

The next minute's opening- and intra-minute features (high, low, average) are not trained on / learned; IEX publishes each minute's data about halfway through to the next minute, so predicting the next minute's closing price is a good goal for the purposes of this example.

In [26]:
grouped = lg.groupby(to_date)

def get_next_close(df):
    # the next minute's closing price is a combination of:
    # - the next minute's "open" field (log-fold change over current-minute close)
    # - the next minute's "close" field (log-fold change over next-minute open)
    df['next_close'] = df.open.shift(-1) + df.close.shift(-1)
    return df
lg = grouped.apply(get_next_close)
lg

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,0.000000,-0.005205,0.000000,-0.005468,-0.002382,6.633250,4.596955,0.001729
2019-04-01 09:31:00,0.000262,0.001467,0.001467,-0.000315,0.000320,5.656854,4.629253,-0.000262
2019-04-01 09:32:00,0.000419,-0.000681,0.000157,-0.001467,-0.000571,5.477226,4.332048,-0.000524
2019-04-01 09:33:00,0.000262,-0.000786,0.000157,-0.001153,-0.000178,5.196152,4.430817,-0.000681
2019-04-01 09:34:00,-0.000367,-0.000315,0.000000,-0.000839,-0.000493,3.464102,4.500735,0.000786
...,...,...,...,...,...,...,...,...
2019-08-01 09:56:00,0.000023,0.000489,0.000815,0.000000,0.000307,6.403124,4.582226,0.000791
2019-08-01 09:57:00,0.000047,0.000745,0.000838,-0.000140,0.000465,6.164414,4.390543,-0.000233
2019-08-01 09:58:00,0.000093,-0.000326,0.000279,-0.000326,0.000135,6.855655,4.761992,0.000047


`next_close` was populated within each day above; that means the final minute of each day is missing a label, so we drop them:

In [27]:
lg.dropna(how='any', inplace=True)
lg.count()

open          30578
close         30578
hi            30578
lo            30578
avg           30578
trades        30578
avgVol        30578
next_close    30578
dtype: int64

## Persistent assignment of labeled samples to {training, validation} sets
Assigning samples to validation vs training should be persistent; we will train iteratively over time on the full history up to that point, and samples marked for validation should always be held out from training.

We achieve this by seeding a PRNG for each sample based on that sample's minute ID (`int(strftime('%Y%m%d%H%M'))`), and pulling a single random float. The set of samples whose floats are less than $V$ should represent $100V$% of the data, and can be used collectively as a validation-split representing proportion $V$ of the total set of labeled samples.

In [28]:
from random import random, seed
validation_split = 0.2

# decide whether a given minute should be treated as a validation (otherwise: training) test case
# seed PRNG so that this info is stable over time, and we never train on samples marked for "validation"
def get_validation_rand(dt):
    seed(int(dt.strftime('%Y%m%d%H%M')))
    return random()

datetime = lg.index.to_series()  # update index since we dropped the last minute of each day
vrs = datetime.apply(get_validation_rand).rename('validation_rand')
vfs = (vrs < validation_split).rename('validation?')
tn, vn = vfs[~vfs].count(), vfs[vfs].count()
n = tn + vn
tf, vf = tn / n, vn / n
print('%d training samples (%.2f%%), %d validation (%.2f%%)' % (tn, 100 * tf, vn, 100 * vf))

24360 training samples (79.67%), 6218 validation (20.33%)


Separate the log-transformed data into training and validation sets.

Also compute {$\mu$, $\sigma$} of the training sets, which will be used to standardize the data.

In [29]:
lg_t = lg[~vfs]
means = lg_t.mean(axis=0).rename('μ')
stddevs = lg_t.std(axis=0).rename('σ')
concat([ means, stddevs ], axis=1)

,μ,σ
open,1.027963e-05,0.000199
close,-7.768920e-06,0.000583
hi,2.871546e-04,0.000421
lo,-3.035063e-04,0.000463
avg,-1.809989e-06,0.000366
trades,4.039914e+00,1.704500
avgVol,4.496012e+00,0.404701
next_close,-8.251628e-07,0.000604


Helpers for showing descriptive statistics of each column of a DataFrame:

In [30]:
def desc(df):
    stats = DF([ describe(df[col]) for col in df.columns ]).set_index(df.columns)
    return desc_df(stats)

def desc_df(stats):
    stats['min'] = stats.minmax.apply(lambda t: t[0])
    stats['max'] = stats.minmax.apply(lambda t: t[1])
    stats['σ'] = sqrt(stats.variance)
    stats.rename(columns={ 'mean': 'μ', 'nobs': 'n' }, inplace=True)
    stats.drop(columns=[ 'minmax', 'variance' ], inplace=True)
    cs = list(stats.columns)
    stats = stats[cs[:2] + [ 'σ' ] + cs[2:-1]]
    return stats

In [31]:
desc(lg)

,n,μ,σ,skewness,kurtosis,min,max
open,30578,1.002744e-05,0.000199,0.019562,5.404455,-0.002417,0.001974
close,30578,-6.704045e-06,0.000575,-0.920087,54.075943,-0.018992,0.009130
hi,30578,2.869340e-04,0.000420,4.252302,36.806992,0.000000,0.009336
lo,30578,-3.017802e-04,0.000453,-8.754134,287.759067,-0.023854,0.000000
avg,30578,-9.964208e-07,0.000360,-2.223047,118.005609,-0.015078,0.005855
trades,30578,4.037606e+00,1.707964,1.323137,4.148110,0.000000,21.679483
avgVol,30578,4.494934e+00,0.407469,-3.796551,45.024909,0.000000,7.509961
next_close,30578,1.506654e-06,0.000600,-1.142262,44.320374,-0.018822,0.007816


The log-transform has captures the idea that, intuitively, a price is as likely to double as half. 

However, it's worth noting that the high kurtosis values imply that the data is not perfectly *normally distributed* after the log-transform; price movements are empirically "heavy-tailed", as we see here.

## Standardize data
Subtract and divide out the mean and stddev of the training features.

Note that:
- the training {$\mu$,$\sigma$} will be used for validation samples as well
- we store $-\mu/\sigma$ (pre-standardization $0$) in `zeros`, for use padding sliding windows later

In [32]:
normed = lg.copy()
zeros = -means / stddevs; zeros

for col in normed.columns:
    normed[col] = (normed[col] - means[col]) / stddevs[col]

trn = normed[~vfs]
val = normed[ vfs]
normed

,open,close,hi,lo,avg,trades,avgVol,next_close
datetime,,,,,,,,
2019-04-01 09:30:00,-0.051703,-8.913373,-0.682214,-11.149967,-6.500041,1.521464,0.249426,2.865734
2019-04-01 09:31:00,1.267194,2.529400,2.803515,-0.024125,0.878258,0.948630,0.329233,-0.432310
2019-04-01 09:32:00,2.054721,-1.153908,-0.309203,-2.511184,-1.553326,0.843245,-0.405147,-0.866328
2019-04-01 09:33:00,1.265260,-1.334117,-0.309046,-1.833304,-0.481231,0.678344,-0.161094,-1.127316
2019-04-01 09:34:00,-1.896997,-0.526147,-0.682214,-1.156398,-1.340947,-0.337819,0.011672,1.303625
...,...,...,...,...,...,...,...,...
2019-08-01 09:55:00,0.417303,1.531735,2.195901,0.655297,2.231919,1.565438,0.934855,0.849799
2019-08-01 09:56:00,0.065438,0.851849,1.253603,0.655297,0.844281,1.386454,0.213032,1.311725
2019-08-01 09:57:00,0.182456,1.290230,1.307823,0.353727,1.275886,1.246406,-0.260609,-0.383925


In [33]:
desc(normed)

,n,μ,σ,skewness,kurtosis,min,max
open,30578,-0.001268,0.999390,0.019562,5.404455,-12.207524,9.877919
close,30578,0.001826,0.986841,-0.920087,54.075943,-32.554993,15.669810
hi,30578,-0.000524,0.996702,4.252302,36.806992,-0.682214,21.497438
lo,30578,0.003727,0.977146,-8.754134,287.759067,-50.848302,0.655297
avg,30578,0.002222,0.981712,-2.223047,118.005609,-41.167308,15.993808
trades,30578,-0.001354,1.002032,1.323137,4.148110,-2.370146,10.348823
avgVol,30578,-0.002662,1.006839,-3.796551,45.024909,-11.109455,7.447342
next_close,30578,0.003862,0.993841,-1.142262,44.320374,-31.171880,12.947016


We continue to observe the presence of heavy outliers (≈10s of standard deviations, as evidenced by the `min`/`max` values in many columns). 

Dropping outliers seems unwise (sudden large price movements are arguably most important to be aware of!), so we'll proceed without further corrections.

Separate the `next_close` ("label") column, and name it `y`:

In [34]:
y = normed.next_close
normed.drop(columns='next_close', inplace=True)
normed

,open,close,hi,lo,avg,trades,avgVol
datetime,,,,,,,
2019-04-01 09:30:00,-0.051703,-8.913373,-0.682214,-11.149967,-6.500041,1.521464,0.249426
2019-04-01 09:31:00,1.267194,2.529400,2.803515,-0.024125,0.878258,0.948630,0.329233
2019-04-01 09:32:00,2.054721,-1.153908,-0.309203,-2.511184,-1.553326,0.843245,-0.405147
2019-04-01 09:33:00,1.265260,-1.334117,-0.309046,-1.833304,-0.481231,0.678344,-0.161094
2019-04-01 09:34:00,-1.896997,-0.526147,-0.682214,-1.156398,-1.340947,-0.337819,0.011672
...,...,...,...,...,...,...,...
2019-08-01 09:55:00,0.417303,1.531735,2.195901,0.655297,2.231919,1.565438,0.934855
2019-08-01 09:56:00,0.065438,0.851849,1.253603,0.655297,0.844281,1.386454,0.213032
2019-08-01 09:57:00,0.182456,1.290230,1.307823,0.353727,1.275886,1.246406,-0.260609


## Rolling window, numpy `ndarray` conversion
Create a numpy array from a rolling 30-minute window of the standardized features:

In [35]:
window = 30

cols = normed.columns
x = np.moveaxis(
    array([ 
        concat(
            [ 
                normed[col].shift(i)
                for i in reversed(range(window)) 
            ], 
            axis=1
        ) \
        .fillna(zeros[col]) \
        .values
        for col in cols
    ]),
    0, 2
)
x.shape    

(30578, 30, 7)

For each minute, we have the last 30-minutes of data about the 7 features.

Partial windows are padded with "0"s (pre-standardization 0; actually $-\mu/\sigma$ in post-standardization space, taken from the `zeros` array above)

Store training/validation data, and assign the full `x` and `y` datasets as a concatenation of training and validation samples, respectively.

In [36]:
tx = x[~vfs]
vx = x[ vfs]
ty = y[~vfs].to_numpy()
vy = y[ vfs].to_numpy()
[ a.shape for a in [ tx,ty, vx,vy, x,y ] ]

[(24360, 30, 7), (24360,), (6218, 30, 7), (6218,), (30578, 30, 7), (30578,)]

In [37]:
factor = exp(y * stddevs.next_close + means.next_close).rename('factor')
next_close = (aapl.close * factor).rename('next_close')
concat([ aapl, next_close, factor ], axis=1)

,open,close,high,low,average,volume,notional,numberOfTrades,prev_close,next_close,factor
datetime,,,,,,,,,,,
2019-04-01 09:30:00,191.645,190.65,191.645,190.600,191.189,4320,825935.940,44,191.645,190.98,1.001731
2019-04-01 09:31:00,190.700,190.98,190.980,190.640,190.761,3246,619210.510,32,190.650,190.93,0.999738
2019-04-01 09:32:00,191.060,190.93,191.090,190.780,190.951,2253,430211.740,30,190.980,190.83,0.999476
2019-04-01 09:33:00,190.980,190.83,191.010,190.760,190.946,2241,427911.290,27,190.930,190.70,0.999319
2019-04-01 09:34:00,190.760,190.70,190.760,190.600,190.666,1069,203822.465,12,190.830,190.85,1.000787
...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01 09:56:00,214.685,214.79,214.860,214.685,214.751,3966,851703.860,41,214.680,214.96,1.000791
2019-08-01 09:57:00,214.800,214.96,214.980,214.770,214.900,3028,650717.425,38,214.790,214.91,0.999767
2019-08-01 09:58:00,214.980,214.91,215.040,214.910,215.009,5451,1172014.895,47,214.960,214.92,1.000047


## Build model

In [38]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [39]:
def create_model():
    model = Sequential()
    model.add(SimpleRNN(4, input_shape=(window, len(cols))))
    model.add(Dense(1))
    model.build()
    model.compile(loss='mae', optimizer='adam')
    return model

In [40]:
import re
regex = '^\d{4}-\d\d-\d\dT\d\d:\d\d(?:_\d{4,})?.ckpt$'
models = sorted([
    dir
    for dir in models_dir.iterdir() 
    if re.match(regex, dir.name)
])
if models:
    ckpt = models[-1]
    print('Loading model from %s' % ckpt)
    model = load_model(str(ckpt))
else:
    print('No pre-existing models found; using untrained model')
    model = create_model()

model.summary()

No pre-existing models found; using untrained model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 4)                 48        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


As a baseline for evaluating our model's training progress, compute the mean absolute error we'd expect from a network that simply learned to predict the mean of the training labels for every sample:

In [41]:
mean(abs(ty - mean(ty))), mean(abs(vy - mean(vy)))

(0.6508239909124275, 0.6373937431098421)

In [42]:
def diffs_df(model):
    def stats(x, y, name=None, predict=True):
        suffix = ' (%s)' % name if name else ''
        if predict:
            predx = model.predict(x)[:, 0]
        else:
            predx = x

        diffs = (y - predx)
        mae = abs(diffs)
        mse = mae**2

        return DF({
            ('y^' + suffix): predx,
            ('Δ' + suffix): diffs,
            ('|Δ|' + suffix): mae,
            ('Δ²' + suffix): mse,
        })

    train = stats(tx, ty, 't')
    val = stats(vx, vy, 'v')
    predx_t = train['y^ (t)']
    predx_v = val['y^ (v)']
    predx = concat([ predx_t, predx_v ])
    all = stats(predx, np.concatenate([ ty, vy ]), name='*', predict=False)

    return concat([
        desc(train),
        desc(val),
        desc(all)
    ])

In [43]:
diffs_df(model)

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),24360,0.081158,0.543712,-0.273228,-1.302285,-1.127019e+00,1.185293
Δ (t),24360,-0.081158,1.137633,-0.510861,27.956164,-3.053473e+01,12.028514
|Δ| (t),24360,0.822517,0.790085,4.989367,102.367542,6.947894e-05,30.534734
Δ² (t),24360,1.300743,7.102967,96.518940,12231.028383,4.827323e-09,932.369999
y^ (v),6218,0.079702,0.544012,-0.281580,-1.308155,-1.084793e+00,1.120143
Δ (v),6218,-0.060710,1.110025,0.239085,8.549961,-1.345827e+01,10.315575
|Δ| (v),6218,0.804857,0.766775,3.301423,25.887396,1.130682e-04,13.458269
Δ² (v),6218,1.235643,3.994149,21.961379,792.259542,1.278441e-08,181.124992
y^ (*),30578,0.080862,0.543765,-0.274929,-1.303462,-1.127019e+00,1.185293
Δ (*),30578,-0.077000,1.132086,-0.367589,24.327143,-3.053473e+01,12.028514


In [44]:
model_ckpt_path = models_dir / run_time.strftime('%Y-%m-%dT%H:%M_{epoch:04d}.ckpt'); model_ckpt_path

PosixPath('/Users/ryan/c/pipelines/notebooks/models/2019-08-01T10:00_{epoch:04d}.ckpt')

In [45]:
%%time
model.fit(tx, ty, 
          validation_data=(vx, vy),
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[
              EarlyStopping(patience=100, restore_best_weights=True),
              ModelCheckpoint(str(model_ckpt_path), save_weights_only=True, save_best_only=True, period=20),
          ]
         )
diffs_df(model)

Train on 24360 samples, validate on 6218 samples
Epoch 1/10000
24360/24360 [==============================] - 1s 52us/sample - loss: 0.8225 - val_loss: 0.8031
Epoch 2/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.8206 - val_loss: 0.8013
Epoch 3/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.8187 - val_loss: 0.7995
Epoch 4/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.8168 - val_loss: 0.7978
Epoch 5/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.8150 - val_loss: 0.7960
Epoch 6/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.8132 - val_loss: 0.7944
Epoch 7/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.8114 - val_loss: 0.7927
Epoch 8/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.8096 - val_loss: 0.7911
Epoch 9/10000
24360/24360 [==============================] - 0s 6us/sample - l

Epoch 74/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.7349 - val_loss: 0.7229
Epoch 75/10000
24360/24360 [==============================] - 0s 5us/sample - loss: 0.7342 - val_loss: 0.7221
Epoch 76/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.7335 - val_loss: 0.7214
Epoch 77/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.7327 - val_loss: 0.7207
Epoch 78/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.7320 - val_loss: 0.7200
Epoch 79/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.7313 - val_loss: 0.7193
Epoch 80/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.7306 - val_loss: 0.7186
Epoch 81/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.7299 - val_loss: 0.7179
Epoch 82/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.7292 - val_loss: 0.7172
Epoch 83/1

24360/24360 [==============================] - 0s 6us/sample - loss: 0.6930 - val_loss: 0.6814
Epoch 149/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6926 - val_loss: 0.6810
Epoch 150/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6922 - val_loss: 0.6806
Epoch 151/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6918 - val_loss: 0.6802
Epoch 152/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6914 - val_loss: 0.6798
Epoch 153/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6910 - val_loss: 0.6794
Epoch 154/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6906 - val_loss: 0.6790
Epoch 155/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6902 - val_loss: 0.6786
Epoch 156/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6898 - val_loss: 0.6782
Epoch 157/10000
2

Epoch 222/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6710 - val_loss: 0.6588
Epoch 223/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6708 - val_loss: 0.6586
Epoch 224/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6706 - val_loss: 0.6584
Epoch 225/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6704 - val_loss: 0.6582
Epoch 226/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6702 - val_loss: 0.6580
Epoch 227/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6700 - val_loss: 0.6578
Epoch 228/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6698 - val_loss: 0.6575
Epoch 229/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6696 - val_loss: 0.6573
Epoch 230/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6694 - val_loss: 0.6571
E

Epoch 296/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6603 - val_loss: 0.6474
Epoch 297/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6602 - val_loss: 0.6473
Epoch 298/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6601 - val_loss: 0.6472
Epoch 299/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6600 - val_loss: 0.6471
Epoch 300/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6599 - val_loss: 0.6470
Epoch 301/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6598 - val_loss: 0.6469
Epoch 302/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6597 - val_loss: 0.6468
Epoch 303/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6596 - val_loss: 0.6467
Epoch 304/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6595 - val_loss: 0.6466
E

...

Epoch 1477/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1478/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1479/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1480/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1481/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1482/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1483/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1484/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1485/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss:

24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1551/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1552/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1553/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1554/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1555/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1556/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1557/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1558/10000
24360/24360 [==============================] - 0s 8us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 155

Epoch 1624/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1625/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1626/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1627/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1628/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1629/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1630/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1631/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6502 - val_loss: 0.6361
Epoch 1632/10000
24360/24360 [==============================] - 0s 7us/sample - loss: 0.6502 - val_loss:

24360/24360 [==============================] - 0s 7us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1698/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1699/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1700/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1701/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1702/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1703/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1704/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 1705/10000
24360/24360 [==============================] - 0s 6us/sample - loss: 0.6501 - val_loss: 0.6361
Epoch 170

,n,μ,σ,skewness,kurtosis,min,max
y^ (t),24360,0.019619,0.043864,1.012658,2.502317,-1.897469e-01,0.406207
Δ (t),24360,-0.019619,0.999297,-1.415224,52.017261,-3.140312e+01,12.819992
|Δ| (t),24360,0.650158,0.759115,6.019415,136.992603,1.741015e-05,31.403115
Δ² (t),24360,0.998938,7.346811,103.074363,13393.219163,3.031135e-10,986.155661
y^ (v),6218,0.018944,0.043306,1.023321,2.396901,-1.155139e-01,0.374952
Δ (v),6218,0.000048,0.969040,-0.374125,14.374930,-1.284923e+01,10.736257
|Δ| (v),6218,0.636099,0.730993,3.877348,32.043209,2.407221e-05,12.849234
Δ² (v),6218,0.938887,3.799591,22.245721,758.895324,5.794713e-10,165.102824
y^ (*),30578,0.019482,0.043751,1.014934,2.482415,-1.897469e-01,0.406207
Δ (*),30578,-0.015620,0.993235,-1.219416,45.123572,-3.140312e+01,12.819992


In [ ]:
hist = DF(model.history.history)
hist.iloc[-110:-90]

In [ ]:
hist.iloc[-200:]

In [ ]:
w = model.get_weights(); w, [ l.shape for l in w ]

In [ ]:
describe(ty)